In [1]:
from astropy.io import fits
import numpy as np
import os.path
import matplotlib.pyplot as plt
from astropy import wcs
from astropy.nddata import Cutout2D
from astropy import coordinates
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
import matplotlib.gridspec as gridspec
import aplpy
import pandas as pd
%matplotlib inline
from photutils import CircularAperture
from photutils import SkyCircularAperture
from photutils import aperture_photometry

In [2]:
part = 3
if part == 0:
    #create catalogues
    path = '/Users/justin/Documents/Master_Thesis/data/catalogues/chandra_7ms/big_table.fits'
    big_table = fits.open(path)[1].data
    flux_cat = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    for i in range(len(big_table)):
        if (big_table.field('Visual_Flag')[i] > 3):
            flux_cat[0].append(big_table.field('ID')[i])
            if (i == 194) | (i == 637):
                flux_cat[1].append(-99)
                flux_cat[2].append(0)
            else:
                flux_cat[1].append(big_table.field('GOODS_b_JanskyAper')[i])
                flux_cat[2].append(big_table.field('GOODS_b_JanskyErrAper')[i])
            flux_cat[3].append(big_table.field('GOODS_v_JanskyAper')[i])
            flux_cat[4].append(big_table.field('GOODS_v_JanskyErrAper')[i])
            flux_cat[5].append(big_table.field('GOODS_i_JanskyAper')[i])
            flux_cat[6].append(big_table.field('GOODS_i_JanskyErrAper')[i])
            flux_cat[7].append(big_table.field('GOODS_z_JanskyAper')[i])
            flux_cat[8].append(big_table.field('GOODS_z_JanskyErrAper')[i])
            if big_table.field('CANDELS_105_Usable')[i] == 0:
                flux_cat[9].append(-99)
                flux_cat[10].append(0)
            else:
                flux_cat[9].append(big_table.field('CANDELS_105_JanskyAper')[i])
                flux_cat[10].append(big_table.field('CANDELS_105_JanskyErrAper')[i])
            if big_table.field('CANDELS_125_Usable')[i] == 0:
                flux_cat[11].append(-99)
                flux_cat[12].append(0)
            else:
                flux_cat[11].append(big_table.field('CANDELS_125_JanskyAper')[i])
                flux_cat[12].append(big_table.field('CANDELS_125_JanskyErrAper')[i])
            if (i==100):
                flux_cat[13].append(-99)
                flux_cat[14].append(0)
            else:
                flux_cat[13].append(big_table.field('CANDELS_160_JanskyAper')[i])
                if (i==716):
                    tf4=(0.25*big_table.field('CANDELS_160_JanskyAper')[i])+big_table.field('CANDELS_160_JanskyErrAper')[i]
                    flux_cat[14].append(tf4)
                else:
                    flux_cat[14].append(big_table.field('CANDELS_160_JanskyErrAper')[i])
            if (i==184) | (i==395) | (i==639) | (i==680) | (i==811):
                flux_cat[15].append(big_table.field('HUGS_JanskyAper')[i])
                tf3=(0.50*big_table.field('HUGS_JanskyAper')[i])+big_table.field('HUGS_JanskyErrAper')[i]
                flux_cat[16].append(tf3)
            else:
                flux_cat[15].append(big_table.field('HUGS_JanskyAper')[i])
                flux_cat[16].append(big_table.field('HUGS_JanskyErrAper')[i])
            flux_cat[17].append(big_table.field('Spitzer_ch1_JanskyAper')[i])
            tf1=(0.15*big_table.field('Spitzer_ch1_JanskyAper')[i])+big_table.field('Spitzer_ch1_JanskyErrAper')[i]
            flux_cat[18].append(tf1)
            flux_cat[19].append(big_table.field('Spitzer_ch2_JanskyAper')[i])
            tf2=(0.15*big_table.field('Spitzer_ch2_JanskyAper')[i])+big_table.field('Spitzer_ch2_JanskyErrAper')[i]
            flux_cat[20].append(tf2)

    ar = np.array(flux_cat)
    at = np.transpose(ar)
    df = pd.DataFrame(data=at, index=None, columns=['id','F233','E233','F236','E236','F238','E238','F240','E240','F202','E202','F203','E203','F205','E205','F269','E269','F18','E18','F19','E19'])
    df['id'] = df['id'].astype('int64')
    path = '/Users/justin/Documents/Master_Thesis/softwares/eazy/inputs/test_cat.cat'
    df.to_csv(path_or_buf=path, sep=' ', index=False)
    path = "sed -i '' '/id/s/^/# /g' ../softwares/eazy/inputs/test_cat.cat"
    os.system(path)

    


In [3]:
if part == 1: #classic sed+chi2 together, all sources
    path = '/Users/justin/Documents/Master_Thesis/data/catalogues/chandra_7ms/big_table.fits'
    big_table = fits.open(path)[1].data
    error_lw = 1.
    error_w = 0.03
    for i in range(len(big_table)):
        if big_table.field('Visual_Flag')[i] > 3:
            path_sed = '/Users/justin/Documents/Master_Thesis/softwares/eazy/inputs/OUTPUT/photz_' + str(i+1) + '.temp_sed'
            path_flx = '/Users/justin/Documents/Master_Thesis/softwares/eazy/inputs/OUTPUT/photz_' + str(i+1) + '.obs_sed'
            path_chi = '/Users/justin/Documents/Master_Thesis/softwares/eazy/inputs/OUTPUT/photz_' + str(i+1) + '.pz'
            cat_sed = pd.read_csv(path_sed, delim_whitespace=True, header=None, names=['Wavelength', 'Flux'], skiprows=[0,1])
            cat_flx = pd.read_csv(path_flx, delim_whitespace=True, header=None, names=['Wavelength', 'FluxCat', 'FluxErrCat', 'FluxErrTot', 'FluxTemp'], skiprows=1)
            cat_chi = pd.read_csv(path_chi, delim_whitespace=True, header=None, names=['z', 'Chi^2'], skiprows=1)

            jy_to_mujy = 1e6
            f = plt.figure()
            ax = f.add_subplot(111)
            ax.plot(cat_sed['Wavelength'], cat_sed['Flux']*jy_to_mujy, marker='', lw=0.5, ls='-', color='dimgray')
            plt.axis([3e3, 5e4, 9e-10*jy_to_mujy, 5e-4*jy_to_mujy])
            ax.set_aspect(0.25)
            for j in range(len(cat_flx)):
                if cat_flx['Wavelength'][j] == 4328.683:
                    h1 = plt.errorbar(cat_flx['Wavelength'][j], cat_flx['FluxCat'][j]*jy_to_mujy, xerr=cat_flx['Wavelength'][j]*error_w, yerr=cat_flx['FluxErrTot'][j]*jy_to_mujy, ls='', capsize=0.5, capthick=0.5, elinewidth=error_lw, color='indigo', label='B')
                if (cat_flx['Wavelength'][j] > 5959.323) & (cat_flx['Wavelength'][j] < 5959.325):
                    h2 = plt.errorbar(cat_flx['Wavelength'][j], cat_flx['FluxCat'][j]*jy_to_mujy, xerr=cat_flx['Wavelength'][j]*error_w, yerr=cat_flx['FluxErrTot'][j]*jy_to_mujy, ls='', capsize=0.5, capthick=0.5, elinewidth=error_lw, color='darkblue', label='V')
                if (cat_flx['Wavelength'][j] > 7704.696) & (cat_flx['Wavelength'][j] < 7704.698):
                    h3 = plt.errorbar(cat_flx['Wavelength'][j], cat_flx['FluxCat'][j]*jy_to_mujy, xerr=cat_flx['Wavelength'][j]*error_w, yerr=cat_flx['FluxErrTot'][j]*jy_to_mujy, ls='', capsize=0.5, capthick=0.5, elinewidth=error_lw, color='blue', label='i')
                if cat_flx['Wavelength'][j] == 9.048604e+03:
                    h4 = plt.errorbar(cat_flx['Wavelength'][j], cat_flx['FluxCat'][j]*jy_to_mujy, xerr=cat_flx['Wavelength'][j]*error_w, yerr=cat_flx['FluxErrTot'][j]*jy_to_mujy, ls='', capsize=0.5, capthick=0.5, elinewidth=error_lw, color='darkcyan', label='z')
                if cat_flx['Wavelength'][j] == 1.057723e+04:
                    h5 = plt.errorbar(cat_flx['Wavelength'][j], cat_flx['FluxCat'][j]*jy_to_mujy, xerr=cat_flx['Wavelength'][j]*error_w, yerr=cat_flx['FluxErrTot'][j]*jy_to_mujy, ls='', capsize=0.5, capthick=0.5, elinewidth=error_lw, color='olivedrab', label='Y')
                if cat_flx['Wavelength'][j] == 1.250085e+04:
                    h6 = plt.errorbar(cat_flx['Wavelength'][j], cat_flx['FluxCat'][j]*jy_to_mujy, xerr=cat_flx['Wavelength'][j]*error_w, yerr=cat_flx['FluxErrTot'][j]*jy_to_mujy, ls='', capsize=0.5, capthick=0.5, elinewidth=error_lw, color='yellowgreen', label='J')
                if cat_flx['Wavelength'][j] == 1.541899e+04:
                    h7 = plt.errorbar(cat_flx['Wavelength'][j], cat_flx['FluxCat'][j]*jy_to_mujy, xerr=cat_flx['Wavelength'][j]*error_w, yerr=cat_flx['FluxErrTot'][j]*jy_to_mujy, ls='', capsize=0.5, capthick=0.5, elinewidth=error_lw, color='orange', label='H')
                if cat_flx['Wavelength'][j] == 2.154491e+04:
                    h8 = plt.errorbar(cat_flx['Wavelength'][j], cat_flx['FluxCat'][j]*jy_to_mujy, xerr=cat_flx['Wavelength'][j]*error_w, yerr=cat_flx['FluxErrTot'][j]*jy_to_mujy, ls='', capsize=0.5, capthick=0.5, elinewidth=error_lw, color='orangered', label='K')
                if cat_flx['Wavelength'][j] == 3.563426e+04:
                    h9 = plt.errorbar(cat_flx['Wavelength'][j], cat_flx['FluxCat'][j]*jy_to_mujy, xerr=cat_flx['Wavelength'][j]*error_w, yerr=cat_flx['FluxErrTot'][j]*jy_to_mujy, ls='', capsize=0.5, capthick=0.5, elinewidth=error_lw, color='red', label='3.6mu')
                if cat_flx['Wavelength'][j] == 4.511019e+04:
                    h10 = plt.errorbar(cat_flx['Wavelength'][j], cat_flx['FluxCat'][j]*jy_to_mujy, xerr=cat_flx['Wavelength'][j]*error_w, yerr=cat_flx['FluxErrTot'][j]*jy_to_mujy, ls='', capsize=0.5, capthick=0.5, elinewidth=error_lw, color='firebrick', label='4.5mu')
            if i==100:
                plt.legend(handles=[h1,h2,h3,h4,h8,h9,h10],numpoints=1,fontsize=7,loc='lower right', bbox_to_anchor=(1.25, 0.))
            else:
                plt.legend(handles=[h1,h2,h3,h4,h5,h6,h7,h8,h9,h10],numpoints=1,fontsize=7,loc='lower right', bbox_to_anchor=(1.25, 0.))
            plt.xscale('log')
            plt.yscale('log')
            plt.xlabel('$\lambda$ in $\AA$', size=13)
            plt.ylabel('$F_{\\nu}$ in $\mu$Jy', size=13)
            a = plt.axes([0.23, .67, .25, .27])
            #labels=['B','V','i','z','Y','J','H','K','3.6mu','4.5mu']
            a.plot(cat_chi['z'], cat_chi['Chi^2'], marker='', lw=0.5, ls='-', color='black')
            plt.yscale('log')
            x1,x2,y1,y2 = plt.axis()
            plt.axis((0,11,y1,y2))
            plt.xlabel('$\mathrm{z}$', size=10)
            plt.ylabel('$\mathrm{\chi^2}$', size=10)
            for label in (a.get_xticklabels() + a.get_yticklabels()):
                label.set_fontsize(7)
            path = '/Users/justin/Documents/Master_Thesis/presentation/plots/eazy/log/photz_' + str(i+1) + '.pdf'
            plt.savefig(path, format='pdf', dpi=1000,bbox_inches='tight')
            plt.clf()


In [4]:
if part == 2: #template sed comparison (not very useful)
    temp0_1 = '/Users/justin/Documents/Master_Thesis/softwares/eazy/templates/EAZY_v1.0/eazy_v1.0_sed1.dat'
    temp0_2 = '/Users/justin/Documents/Master_Thesis/softwares/eazy/templates/EAZY_v1.0/eazy_v1.0_sed2.dat'
    temp0_3 = '/Users/justin/Documents/Master_Thesis/softwares/eazy/templates/EAZY_v1.0/eazy_v1.0_sed3.dat'
    temp0_4 = '/Users/justin/Documents/Master_Thesis/softwares/eazy/templates/EAZY_v1.0/eazy_v1.0_sed4.dat'
    temp0_5 = '/Users/justin/Documents/Master_Thesis/softwares/eazy/templates/EAZY_v1.0/eazy_v1.0_sed5.dat'
    temp0_6 = '/Users/justin/Documents/Master_Thesis/softwares/eazy/templates/EAZY_v1.0/eazy_v1.0_sed6.dat'
    temp1_1 = '/Users/justin/Documents/Master_Thesis/softwares/eazy/templates/EAZY_v1.1_lines/eazy_v1.1_sed1.dat'
    temp1_2 = '/Users/justin/Documents/Master_Thesis/softwares/eazy/templates/EAZY_v1.1_lines/eazy_v1.1_sed2.dat'
    temp1_3 = '/Users/justin/Documents/Master_Thesis/softwares/eazy/templates/EAZY_v1.1_lines/eazy_v1.1_sed3.dat'
    temp1_4 = '/Users/justin/Documents/Master_Thesis/softwares/eazy/templates/EAZY_v1.1_lines/eazy_v1.1_sed4.dat'
    temp1_5 = '/Users/justin/Documents/Master_Thesis/softwares/eazy/templates/EAZY_v1.1_lines/eazy_v1.1_sed5.dat'
    temp1_6 = '/Users/justin/Documents/Master_Thesis/softwares/eazy/templates/EAZY_v1.1_lines/eazy_v1.1_sed6.dat'
    temp1_7 = '/Users/justin/Documents/Master_Thesis/softwares/eazy/templates/EAZY_v1.1_lines/eazy_v1.1_sed7.dat'
    temp1_8 = '/Users/justin/Documents/Master_Thesis/softwares/eazy/templates/Dusty/c09_del_8.6_z_0.019_chab_age09.40_av2.0.dat'
    temp1_9 = '/Users/justin/Documents/Master_Thesis/softwares/eazy/templates/erb2010_highEW.dat'
    temp1_10 = '/Users/justin/Documents/Master_Thesis/softwares/eazy/templates/quasar_template_orig.dat'
    
    sed0_1 = pd.read_csv(temp0_1, delim_whitespace=True, header=None, names=['Wavelength', 'Flux'])
    sed0_2 = pd.read_csv(temp0_2, delim_whitespace=True, header=None, names=['Wavelength', 'Flux'])
    sed0_3 = pd.read_csv(temp0_3, delim_whitespace=True, header=None, names=['Wavelength', 'Flux'])
    sed0_4 = pd.read_csv(temp0_4, delim_whitespace=True, header=None, names=['Wavelength', 'Flux'])
    sed0_5 = pd.read_csv(temp0_5, delim_whitespace=True, header=None, names=['Wavelength', 'Flux'])
    sed0_6 = pd.read_csv(temp0_6, delim_whitespace=True, header=None, names=['Wavelength', 'Flux'])
    sed1_1 = pd.read_csv(temp1_1, delim_whitespace=True, header=None, names=['Wavelength', 'Flux'])
    sed1_2 = pd.read_csv(temp1_2, delim_whitespace=True, header=None, names=['Wavelength', 'Flux'])
    sed1_3 = pd.read_csv(temp1_3, delim_whitespace=True, header=None, names=['Wavelength', 'Flux'])
    sed1_4 = pd.read_csv(temp1_4, delim_whitespace=True, header=None, names=['Wavelength', 'Flux'])
    sed1_5 = pd.read_csv(temp1_5, delim_whitespace=True, header=None, names=['Wavelength', 'Flux'])
    sed1_6 = pd.read_csv(temp1_6, delim_whitespace=True, header=None, names=['Wavelength', 'Flux'])
    sed1_7 = pd.read_csv(temp1_7, delim_whitespace=True, header=None, names=['Wavelength', 'Flux'])
    sed1_8 = pd.read_csv(temp1_8, delim_whitespace=True, header=None, names=['Wavelength', 'Flux'])
    sed1_9 = pd.read_csv(temp1_9, delim_whitespace=True, header=None, names=['Wavelength', 'Flux'])
    sed1_10 = pd.read_csv(temp1_10, delim_whitespace=True, header=None, usecols=[0,1], names=['Wavelength', 'Flux'])
    #print(sed1_5['Flux'][1879]/sed1_10['Flux'][5200])
    sed1_10['Flux'] = sed1_10['Flux'] * 1.2
    path = '/Users/justin/Documents/Master_Thesis/softwares/eazy/templates/quasar_template.dat'
    sed1_10.to_csv(path_or_buf=path, sep=' ', index=False)
    
    f=plt.figure()
    ax=f.add_subplot(111)
    ax.plot(sed0_1['Wavelength'], sed0_1['Flux']/sed0_1['Flux'][482], label='PEGASE1')
    ax.plot(sed0_2['Wavelength'], sed0_2['Flux']/sed0_2['Flux'][482], label='PEGASE2')
    ax.plot(sed0_3['Wavelength'], sed0_3['Flux']/sed0_3['Flux'][482], label='PEGASE3')
    ax.plot(sed0_4['Wavelength'], sed0_4['Flux']/sed0_4['Flux'][482], label='PEGASE4')
    ax.plot(sed0_5['Wavelength'], sed0_5['Flux']/sed0_5['Flux'][482], label='PEGASE5')
    ax.plot(sed0_6['Wavelength'], sed0_6['Flux']/sed0_6['Flux'][482], label='Young&Dusty6')
    plt.xscale('log')
    plt.yscale('log')
    plt.axis([7e2, 1e5, 5e-4, 50])
    plt.legend(loc='upper right', bbox_to_anchor=(1.1, 1.1), fontsize=10)
    plt.xlabel('$\lambda$ in $\AA$', size=13)
    plt.ylabel('$F_{\\lambda}/F_{6000}$', size=13)
    path = '/Users/justin/Documents/Master_Thesis/presentation/plots/templates1.0.pdf'
    plt.savefig(path, format='pdf', dpi=1000,bbox_inches='tight')
    g=plt.figure()
    ax2=g.add_subplot(111)
    ax2.plot(sed1_1['Wavelength'], sed1_1['Flux']/sed1_1['Flux'][1879], label='PEGASE1')
    ax2.plot(sed1_2['Wavelength'], sed1_2['Flux']/sed1_2['Flux'][1879], label='PEGASE2')
    ax2.plot(sed1_3['Wavelength'], sed1_3['Flux']/sed1_3['Flux'][1879], label='PEGASE3')
    ax2.plot(sed1_4['Wavelength'], sed1_4['Flux']/sed1_4['Flux'][1879], label='PEGASE4')
    ax2.plot(sed1_5['Wavelength'], sed1_5['Flux']/sed1_5['Flux'][1879], label='PEGASE5')
    ax2.plot(sed1_6['Wavelength'], sed1_6['Flux']/sed1_6['Flux'][1879], label='Young&Dusty6')
    ax2.plot(sed1_7['Wavelength'], sed1_7['Flux']/sed1_7['Flux'][261], label='Maraston7')
    ax2.plot(sed1_8['Wavelength'], sed1_8['Flux']/sed1_8['Flux'][483], label='DUSTY8')
    ax2.plot(sed1_9['Wavelength'], sed1_9['Flux']/sed1_9['Flux'][985], color='orange', label='highEW9')
    ax2.plot(sed1_10['Wavelength'], sed1_10['Flux']/sed1_10['Flux'][5200], color='brown', label='SDSS quasar')
    plt.xscale('log')
    plt.yscale('log')
    plt.legend(loc='upper right', bbox_to_anchor=(1.1, 1.1), fontsize=10)
    plt.axis([7e2, 1e5, 5e-4, 200])
    plt.xlabel('$\lambda$ in $\AA$', size=13)
    plt.ylabel('$F_{\\lambda}/F_{6000}$', size=13)
    path = '/Users/justin/Documents/Master_Thesis/presentation/plots/templates1.1.pdf'
    plt.savefig(path, format='pdf', dpi=1000,bbox_inches='tight')
    

In [5]:
if part == 3: #separate sed and chi2, for all sources (most recent version)
    path = '/Users/justin/Documents/Master_Thesis/data/catalogues/chandra_7ms/big_table.fits'
    big_table = fits.open(path)[1].data
    error_lw = 1.
    error_w = 0.03
    path = '/Users/justin/Documents/Master_Thesis/softwares/eazy/inputs/OUTPUT/photz.zout'
    cat_z = pd.read_csv(path, delim_whitespace=True, header=None, names=['ID', 'z_spec', 'z_a','f','chi2','l1','u1','l2','u2','l3','u3','3f','s2s','gt4','4f','bt'], skiprows=[0,1])

    for i in range(len(big_table)):
        if big_table.field('Visual_Flag')[i] > 3:
            path_sed = '/Users/justin/Documents/Master_Thesis/softwares/eazy/inputs/OUTPUT/photz_' + str(i+1) + '.temp_sed'
            path_flx = '/Users/justin/Documents/Master_Thesis/softwares/eazy/inputs/OUTPUT/photz_' + str(i+1) + '.obs_sed'
            path_chi = '/Users/justin/Documents/Master_Thesis/softwares/eazy/inputs/OUTPUT/photz_' + str(i+1) + '.pz'
            cat_sed = pd.read_csv(path_sed, delim_whitespace=True, header=None, names=['Wavelength', 'Flux'], skiprows=[0,1])
            cat_flx = pd.read_csv(path_flx, delim_whitespace=True, header=None, names=['Wavelength', 'FluxCat', 'FluxErrCat', 'FluxErrTot', 'FluxTemp'], skiprows=1)
            cat_chi = pd.read_csv(path_chi, delim_whitespace=True, header=None, names=['z', 'Chi^2'], skiprows=1)

            jy_to_mujy = 1e6
            f = plt.figure(figsize=(4,4))
            ax = f.add_subplot(111)
            ax.plot(cat_sed['Wavelength'], cat_sed['Flux']*jy_to_mujy, marker='', lw=0.5, ls='-', color='dimgray')
            plt.axis([3e3, 5e4, 9e-10*jy_to_mujy, 5e-4*jy_to_mujy])
            #ax.set_aspect(0.25)
            for j in range(len(cat_flx)):
                if cat_flx['Wavelength'][j] == 4328.683:
                    h1 = plt.errorbar(cat_flx['Wavelength'][j], cat_flx['FluxCat'][j]*jy_to_mujy, xerr=cat_flx['Wavelength'][j]*error_w, yerr=cat_flx['FluxErrTot'][j]*jy_to_mujy, ls='', capsize=0.5, capthick=0.5, elinewidth=error_lw, color='indigo', label='B')
                if (cat_flx['Wavelength'][j] > 5959.323) & (cat_flx['Wavelength'][j] < 5959.325):
                    h2 = plt.errorbar(cat_flx['Wavelength'][j], cat_flx['FluxCat'][j]*jy_to_mujy, xerr=cat_flx['Wavelength'][j]*error_w, yerr=cat_flx['FluxErrTot'][j]*jy_to_mujy, ls='', capsize=0.5, capthick=0.5, elinewidth=error_lw, color='darkblue', label='V')
                if (cat_flx['Wavelength'][j] > 7704.696) & (cat_flx['Wavelength'][j] < 7704.698):
                    h3 = plt.errorbar(cat_flx['Wavelength'][j], cat_flx['FluxCat'][j]*jy_to_mujy, xerr=cat_flx['Wavelength'][j]*error_w, yerr=cat_flx['FluxErrTot'][j]*jy_to_mujy, ls='', capsize=0.5, capthick=0.5, elinewidth=error_lw, color='blue', label='i')
                if cat_flx['Wavelength'][j] == 9.048604e+03:
                    h4 = plt.errorbar(cat_flx['Wavelength'][j], cat_flx['FluxCat'][j]*jy_to_mujy, xerr=cat_flx['Wavelength'][j]*error_w, yerr=cat_flx['FluxErrTot'][j]*jy_to_mujy, ls='', capsize=0.5, capthick=0.5, elinewidth=error_lw, color='darkcyan', label='z')
                if cat_flx['Wavelength'][j] == 1.057723e+04:
                    h5 = plt.errorbar(cat_flx['Wavelength'][j], cat_flx['FluxCat'][j]*jy_to_mujy, xerr=cat_flx['Wavelength'][j]*error_w, yerr=cat_flx['FluxErrTot'][j]*jy_to_mujy, ls='', capsize=0.5, capthick=0.5, elinewidth=error_lw, color='olivedrab', label='Y')
                if cat_flx['Wavelength'][j] == 1.250085e+04:
                    h6 = plt.errorbar(cat_flx['Wavelength'][j], cat_flx['FluxCat'][j]*jy_to_mujy, xerr=cat_flx['Wavelength'][j]*error_w, yerr=cat_flx['FluxErrTot'][j]*jy_to_mujy, ls='', capsize=0.5, capthick=0.5, elinewidth=error_lw, color='yellowgreen', label='J')
                if cat_flx['Wavelength'][j] == 1.541899e+04:
                    h7 = plt.errorbar(cat_flx['Wavelength'][j], cat_flx['FluxCat'][j]*jy_to_mujy, xerr=cat_flx['Wavelength'][j]*error_w, yerr=cat_flx['FluxErrTot'][j]*jy_to_mujy, ls='', capsize=0.5, capthick=0.5, elinewidth=error_lw, color='orange', label='H')
                if cat_flx['Wavelength'][j] == 2.154491e+04:
                    h8 = plt.errorbar(cat_flx['Wavelength'][j], cat_flx['FluxCat'][j]*jy_to_mujy, xerr=cat_flx['Wavelength'][j]*error_w, yerr=cat_flx['FluxErrTot'][j]*jy_to_mujy, ls='', capsize=0.5, capthick=0.5, elinewidth=error_lw, color='orangered', label='K')
                if cat_flx['Wavelength'][j] == 3.563426e+04:
                    h9 = plt.errorbar(cat_flx['Wavelength'][j], cat_flx['FluxCat'][j]*jy_to_mujy, xerr=cat_flx['Wavelength'][j]*error_w, yerr=cat_flx['FluxErrTot'][j]*jy_to_mujy, ls='', capsize=0.5, capthick=0.5, elinewidth=error_lw, color='red', label='3.6mu')
                if cat_flx['Wavelength'][j] == 4.511019e+04:
                    h10 = plt.errorbar(cat_flx['Wavelength'][j], cat_flx['FluxCat'][j]*jy_to_mujy, xerr=cat_flx['Wavelength'][j]*error_w, yerr=cat_flx['FluxErrTot'][j]*jy_to_mujy, ls='', capsize=0.5, capthick=0.5, elinewidth=error_lw, color='firebrick', label='4.5mu')
            if i==100:
                plt.legend(handles=[h1,h2,h3,h4,h8,h9,h10],frameon=False,numpoints=1,fontsize=7,loc='upper left')
            else:
                plt.legend(handles=[h1,h2,h3,h4,h5,h6,h7,h8,h9,h10],frameon=False,numpoints=1,fontsize=7,loc='upper left')
            plt.xscale('log')
            plt.yscale('log')
            plt.xlabel('$\lambda$ in $\AA$')
            plt.ylabel('$F_{\\nu}$ in $\mu$Jy')
            for j in range(len(cat_z)):
                if cat_z['ID'][j]==i+1:
                    plt.text(s='$z_{phot}='+'{:.2f}'.format(cat_z['z_a'][j])+'$',x=10000, y=200)
            path = '/Users/justin/Documents/Master_Thesis/presentation/plots/eazy/sep/sed_' + str(i+1) + '.pdf'
            plt.savefig(path, format='pdf', dpi=1000,bbox_inches='tight')
            plt.clf()
            
            g=plt.figure(figsize=(4,4))
            ax=g.add_subplot(111)
            plt.plot(cat_chi['z'],cat_chi['Chi^2'], marker='', lw=1., ls='-', color='black',label='')
            #plt.yscale('log')
            x1,x2,y1,y2 = plt.axis()
            plt.axis((0,11,0,20))
            for j in range(len(cat_z)):
                if cat_z['ID'][j]==i+1:
                    chi_l1=np.interp(cat_z['l1'][j],cat_chi['z'],cat_chi['Chi^2'])
                    chi_u1=np.interp(cat_z['u1'][j],cat_chi['z'],cat_chi['Chi^2'])
                    chi_l2=np.interp(cat_z['l2'][j],cat_chi['z'],cat_chi['Chi^2'])
                    chi_u2=np.interp(cat_z['u2'][j],cat_chi['z'],cat_chi['Chi^2'])
                    chi_l3=np.interp(cat_z['l3'][j],cat_chi['z'],cat_chi['Chi^2'])
                    chi_u3=np.interp(cat_z['u3'][j],cat_chi['z'],cat_chi['Chi^2'])
                    chi_best=np.interp(cat_z['z_a'][j],cat_chi['z'],cat_chi['Chi^2'])
                    plt.plot([cat_z['l1'][j],cat_z['l1'][j]], [0,chi_l1], marker='', lw=0.5, ls='-', color='black',label='$1\sigma$')
                    plt.plot([cat_z['l2'][j],cat_z['l2'][j]], [0,chi_l2], marker='', lw=0.5, ls='--', color='black',label='$2\sigma$')
                    plt.plot([cat_z['l3'][j],cat_z['l3'][j]], [0,chi_l3], marker='', lw=0.5, ls='-.', color='black',label='$3\sigma$')
                    plt.plot([cat_z['u1'][j],cat_z['u1'][j]], [0,chi_u1], marker='', lw=0.5, ls='-', color='black')
                    plt.plot([cat_z['u2'][j],cat_z['u2'][j]], [0,chi_u2], marker='', lw=0.5, ls='--', color='black')
                    plt.plot([cat_z['u3'][j],cat_z['u3'][j]], [0,chi_u3], marker='', lw=0.5, ls='-.', color='black')
                    #plt.arrow(cat_z['z_a'][j], chi_best, marker='.', ms=6, ls='', color='red')
                    ax.arrow(cat_z['z_a'][j], chi_best+3, 0., -2,length_includes_head=True, head_width=0.2, head_length=0.2, fc='k', ec='k', lw=1)
            plt.fill_between(cat_chi['z'],0,cat_chi['Chi^2'],color='lightgray')
            plt.xlabel('$\mathrm{z}$')
            plt.ylabel('$\mathrm{\chi^2}$')
            plt.legend(frameon=False, loc='best')
            #for label in (ax.get_xticklabels() + ax.get_yticklabels()):
                #label.set_fontsize(7)
            path = '/Users/justin/Documents/Master_Thesis/presentation/plots/eazy/sep/chi2_' + str(i+1) + '.pdf'
            plt.savefig(path, format='pdf', dpi=1000,bbox_inches='tight')
            plt.clf()
            
            h=plt.figure(figsize=(4,4))
            ax=h.add_subplot(111)
            plt.plot(cat_chi['z'],cat_chi['Chi^2'], marker='', lw=1., ls='-', color='black',label='')
            plt.yscale('log')
            x1,x2,y1,y2 = plt.axis()
            plt.axis((0,11,y1,y2))
            for j in range(len(cat_z)):
                if cat_z['ID'][j]==i+1:
                    chi_l1=np.interp(cat_z['l1'][j],cat_chi['z'],cat_chi['Chi^2'])
                    chi_u1=np.interp(cat_z['u1'][j],cat_chi['z'],cat_chi['Chi^2'])
                    chi_l2=np.interp(cat_z['l2'][j],cat_chi['z'],cat_chi['Chi^2'])
                    chi_u2=np.interp(cat_z['u2'][j],cat_chi['z'],cat_chi['Chi^2'])
                    chi_l3=np.interp(cat_z['l3'][j],cat_chi['z'],cat_chi['Chi^2'])
                    chi_u3=np.interp(cat_z['u3'][j],cat_chi['z'],cat_chi['Chi^2'])
                    chi_best=np.interp(cat_z['z_a'][j],cat_chi['z'],cat_chi['Chi^2'])
                    plt.plot([cat_z['l1'][j],cat_z['l1'][j]], [y1,chi_l1], marker='', lw=0.5, ls='-', color='black',label='$1\sigma$')
                    plt.plot([cat_z['l2'][j],cat_z['l2'][j]], [y1,chi_l2], marker='', lw=0.5, ls='--', color='black',label='$2\sigma$')
                    plt.plot([cat_z['l3'][j],cat_z['l3'][j]], [y1,chi_l3], marker='', lw=0.5, ls='-.', color='black',label='$3\sigma$')
                    plt.plot([cat_z['u1'][j],cat_z['u1'][j]], [y1,chi_u1], marker='', lw=0.5, ls='-', color='black')
                    plt.plot([cat_z['u2'][j],cat_z['u2'][j]], [y1,chi_u2], marker='', lw=0.5, ls='--', color='black')
                    plt.plot([cat_z['u3'][j],cat_z['u3'][j]], [y1,chi_u3], marker='', lw=0.5, ls='-.', color='black')
                    #plt.arrow(cat_z['z_a'][j], chi_best, marker='.', ms=6, ls='', color='red')
                    ax.arrow(cat_z['z_a'][j], chi_best*7, 0., -chi_best*4.,length_includes_head=True, head_width=0.2, head_length=chi_best*0.3, fc='k', ec='k', lw=1)
            plt.fill_between(cat_chi['z'],y1,cat_chi['Chi^2'],color='lightgray')
            plt.xlabel('$\mathrm{z}$')
            plt.ylabel('$\mathrm{\chi^2}$')
            plt.legend(frameon=False, loc='best')
            #for label in (ax.get_xticklabels() + ax.get_yticklabels()):
                #label.set_fontsize(7)
            path = '/Users/justin/Documents/Master_Thesis/presentation/plots/eazy/sep/chi2_' + str(i+1) + '_log.pdf'
            plt.savefig(path, format='pdf', dpi=1000,bbox_inches='tight')
            plt.clf()



/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [6]:
if part == 4: #manual photometry, + create catalogue
    path = '/Users/justin/Documents/Master_Thesis/data/catalogues/chandra_7ms/big_table.fits'
    big_table = fits.open(path)[1].data
    error_lw = 1.
    error_w = 0.03
    path = '/Users/justin/Documents/Master_Thesis/softwares/eazy/inputs/OUTPUT/photz.zout'
    cat_z = pd.read_csv(path, delim_whitespace=True, header=None, names=['ID', 'z_spec', 'z_a','f','chi2','l1','u1','l2','u2','l3','u3','3f','s2s','gt4','4f','bt'], skiprows=[0,1])

    flux_cat = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    
    for i in [526,621,639,713]:
        
        z_grid=[3.,3.5,4.,4.5,5.,5.5,6.,6.5,7.,7.5,8.]
        d_l=[25417.2,30575.2,35842.1,41200.5,46637.4,52142.5,57707.7,63326.5,68993.5,74704.0,80454.2] #Mpc luminosity distance based on redshift (Ned Wright cosm. calc.)  
        if i==526:
            wav=[4328.683,5959.323,7704.696,9.048604e+03,1.250085e+04]
            a = ['goods_b', 'goods_v', 'goods_i', 'goods_z', 'candels_125_', 'candels_160_', 'hugs_', 'spitzer_ch1_','spitzer_ch2_']
            b = ['goods_b', 'goods_v', 'goods_i', 'goods_z', 'candels_125', 'candels_160', 'hugs', 'spitzer_ch1','spitzer_ch2']
            c = ['goods', 'goods', 'goods', 'goods', 'candels', 'candels', 'hugs', 'spitzer', 'spitzer']
            fluxratio = [1.95338693e-7,9.238062e-8,1.98796897e-7,4.12286699e-7,1.16912375e-7,1.51876003e-7,9.9999999e-10,3.92276e-6,4.80825e-6]
            zeropoint = [25.673, 26.486, 25.654, 24.862, 26.2303, 25.9463, 31.4, 22.416, 22.195]
        else:
            wav=[4328.683,5959.323,7704.696,9.048604e+03,1.057723e+04,1.250085e+04]
            a = ['goods_b', 'goods_v', 'goods_i', 'goods_z', 'candels_105_', 'candels_125_', 'candels_160_', 'hugs_', 'spitzer_ch1_','spitzer_ch2_']
            b = ['goods_b', 'goods_v', 'goods_i', 'goods_z', 'candels_105', 'candels_125', 'candels_160', 'hugs', 'spitzer_ch1','spitzer_ch2']
            c = ['goods', 'goods', 'goods', 'goods', 'candels', 'candels', 'candels', 'hugs', 'spitzer', 'spitzer']
            fluxratio = [1.95338693e-7,9.238062e-8,1.98796897e-7,4.12286699e-7,1.12851106e-7,1.16912375e-7,1.51876003e-7,9.9999999e-10,3.92276e-6,4.80825e-6]
            zeropoint = [25.673, 26.486, 25.654, 24.862, 26.2687, 26.2303, 25.9463, 31.4, 22.416, 22.195]
        mag = []
        magerr = []
        mag1450 = []
        flux = []
        flux_jy = []
        fluxerr = []
        fluxerr_jy = []
        sn = []
        for j in range(len(wav)):
            path = '/Users/justin/Documents/Master_Thesis/data/surveys/cutouts/' + c[j] + '/cutout_' + a[j] + str(i+1) + '.fits'
            im = fits.open(path)[0]
            path = '/Users/justin/Documents/Master_Thesis/data/catalogues/cutouts/' + c[j] + '/cat_' + a[j] + str(i+1) + '_bkg.fits'
            bkg = fits.open(path)[0]
            path = '/Users/justin/Documents/Master_Thesis/data/surveys/cutouts/' + c[j] + '/cutout_' + a[j] + str(i+1) + 'rms.fits'
            rms = fits.open(path)[0]
            nx = rms.header['NAXIS1']
            ny = rms.header['NAXIS2']
            for x in range(0,nx):
                for y in range(0,ny):
                    rms.data[y,x] = rms.data[y,x]**2.
            position = SkyCoord(big_table.field('CANDELS_160_RA')[i]*u.deg, big_table.field('CANDELS_160_Dec')[i]*u.deg, frame='icrs')
            rad=big_table.field('Aperture')[i]/2.
            circaper = SkyCircularAperture(position, r=rad*u.arcsec)
            phot_table = aperture_photometry(im, circaper)
            phot_table_bkg = aperture_photometry(bkg, circaper)
            phot_table_rms = aperture_photometry(rms, circaper)

            mag.append(zeropoint[j] - (2.5*np.log10(phot_table['aperture_sum'][0]-phot_table_bkg['aperture_sum'][0])))
            flux.append(phot_table['aperture_sum'][0]-phot_table_bkg['aperture_sum'][0])
            flux_jy.append((phot_table['aperture_sum'][0]-phot_table_bkg['aperture_sum'][0])*fluxratio[j])
            fluxerr.append(phot_table_rms['aperture_sum'][0]**0.5)
            fluxerr_jy.append((phot_table_rms['aperture_sum'][0]**0.5)*fluxratio[j])
            sn.append((phot_table['aperture_sum'][0]-phot_table_bkg['aperture_sum'][0])/(phot_table_rms['aperture_sum'][0]**0.5))
            magerr.append(2.5*0.43*fluxerr[j]/flux[j])
            mag1450.append(-2.5*np.log10(flux_jy[j]*((np.interp(big_table.field('Phot_z')[i],z_grid,d_l)*1000000./10.)**2)/3631.))

            print(b[j], flux_jy[j], fluxerr_jy[j], sn[j], '%.2f\pm%.2f' % (mag1450[j], magerr[j]))

        flux_cat[0].append(big_table.field('ID')[i])
        if sn[0] > 3:
            flux_cat[1].append(flux_jy[0])
            flux_cat[2].append(fluxerr_jy[0])
        else:
            flux_cat[1].append(big_table.field('GOODS_b_JanskyAper')[i])
            flux_cat[2].append(big_table.field('GOODS_b_JanskyErrAper')[i])
        if sn[1] > 3:
            flux_cat[3].append(flux_jy[1])
            flux_cat[4].append(fluxerr_jy[1])
        else:
            flux_cat[3].append(big_table.field('GOODS_v_JanskyAper')[i])
            flux_cat[4].append(big_table.field('GOODS_v_JanskyErrAper')[i])
        if sn[2] > 3:
            flux_cat[5].append(flux_jy[2])
            flux_cat[6].append(fluxerr_jy[2])
        else:
            flux_cat[5].append(big_table.field('GOODS_i_JanskyAper')[i])
            flux_cat[6].append(big_table.field('GOODS_i_JanskyErrAper')[i])
        if sn[3] > 3:
            flux_cat[7].append(flux_jy[3])
            flux_cat[8].append(fluxerr_jy[3])
        else:
            flux_cat[7].append(big_table.field('GOODS_z_JanskyAper')[i])
            flux_cat[8].append(big_table.field('GOODS_z_JanskyErrAper')[i])
        if big_table.field('CANDELS_105_Usable')[i] == 0:
            flux_cat[9].append(-99)
            flux_cat[10].append(0)
            if sn[4] > 3:
                flux_cat[11].append(flux_jy[4])
                flux_cat[12].append(fluxerr_jy[4])
            else: 
                flux_cat[11].append(big_table.field('CANDELS_125_JanskyAper')[i])
                flux_cat[12].append(big_table.field('CANDELS_125_JanskyErrAper')[i])
        else:
            if sn[4] > 3:
                flux_cat[9].append(flux_jy[4])
                flux_cat[10].append(fluxerr_jy[4])
            else:
                flux_cat[9].append(big_table.field('CANDELS_105_JanskyAper')[i])
                flux_cat[10].append(big_table.field('CANDELS_105_JanskyErrAper')[i])
            if sn[5] > 3:
                flux_cat[11].append(flux_jy[5])
                flux_cat[12].append(fluxerr_jy[5])
            else:
                flux_cat[11].append(big_table.field('CANDELS_125_JanskyAper')[i])
                flux_cat[12].append(big_table.field('CANDELS_125_JanskyErrAper')[i])
        flux_cat[13].append(big_table.field('CANDELS_160_JanskyAper')[i])
        flux_cat[14].append(big_table.field('CANDELS_160_JanskyErrAper')[i])
        if (i==639):
            flux_cat[15].append(big_table.field('HUGS_JanskyAper')[i])
            tf3=(0.50*big_table.field('HUGS_JanskyAper')[i])+big_table.field('HUGS_JanskyErrAper')[i]
            flux_cat[16].append(tf3)
        else:
            flux_cat[15].append(big_table.field('HUGS_JanskyAper')[i])
            flux_cat[16].append(big_table.field('HUGS_JanskyErrAper')[i])
        flux_cat[17].append(big_table.field('Spitzer_ch1_JanskyAper')[i])
        tf1=(0.15*big_table.field('Spitzer_ch1_JanskyAper')[i])+big_table.field('Spitzer_ch1_JanskyErrAper')[i]
        flux_cat[18].append(tf1)
        flux_cat[19].append(big_table.field('Spitzer_ch2_JanskyAper')[i])
        tf2=(0.15*big_table.field('Spitzer_ch2_JanskyAper')[i])+big_table.field('Spitzer_ch2_JanskyErrAper')[i]
        flux_cat[20].append(tf2)
    
    ar = np.array(flux_cat)
    at = np.transpose(ar)
    df = pd.DataFrame(data=at, index=None, columns=['id','F233','E233','F236','E236','F238','E238','F240','E240','F202','E202','F203','E203','F205','E205','F269','E269','F18','E18','F19','E19'])
    df['id'] = df['id'].astype('int64')
    path = '/Users/justin/Documents/Master_Thesis/softwares/eazy/inputs/test_cat_manual.cat'
    df.to_csv(path_or_buf=path, sep=' ', index=False)
    path = "sed -i '' '/id/s/^/# /g' ../softwares/eazy/inputs/test_cat_manual.cat"
    os.system(path)





In [7]:
if part == 5: #sed + chi2 separate, manual photometry only
    path = '/Users/justin/Documents/Master_Thesis/data/catalogues/chandra_7ms/big_table.fits'
    big_table = fits.open(path)[1].data
    error_lw = 1.
    error_w = 0.03
    path = '/Users/justin/Documents/Master_Thesis/softwares/eazy/inputs/OUTPUT_manual/photz.zout'
    cat_z = pd.read_csv(path, delim_whitespace=True, header=None, names=['ID', 'z_spec', 'z_a','f','chi2','l1','u1','l2','u2','l3','u3','3f','s2s','gt4','4f','bt'], skiprows=[0,1])

    for i in [526,621,639,713]:
        path_sed = '/Users/justin/Documents/Master_Thesis/softwares/eazy/inputs/OUTPUT_manual/photz_' + str(i+1) + '.temp_sed'
        path_flx = '/Users/justin/Documents/Master_Thesis/softwares/eazy/inputs/OUTPUT_manual/photz_' + str(i+1) + '.obs_sed'
        path_chi = '/Users/justin/Documents/Master_Thesis/softwares/eazy/inputs/OUTPUT_manual/photz_' + str(i+1) + '.pz'
        cat_sed = pd.read_csv(path_sed, delim_whitespace=True, header=None, names=['Wavelength', 'Flux'], skiprows=[0,1])
        cat_flx = pd.read_csv(path_flx, delim_whitespace=True, header=None, names=['Wavelength', 'FluxCat', 'FluxErrCat', 'FluxErrTot', 'FluxTemp'], skiprows=1)
        cat_chi = pd.read_csv(path_chi, delim_whitespace=True, header=None, names=['z', 'Chi^2'], skiprows=1)

        jy_to_mujy = 1e6
        f = plt.figure(figsize=(4,4))
        ax = f.add_subplot(111)
        ax.plot(cat_sed['Wavelength'], cat_sed['Flux']*jy_to_mujy, marker='', lw=0.5, ls='-', color='dimgray')
        plt.axis([3e3, 5e4, 9e-10*jy_to_mujy, 5e-4*jy_to_mujy])
        #ax.set_aspect(0.25)
        for j in range(len(cat_flx)):
            if cat_flx['Wavelength'][j] == 4328.683:
                h1 = plt.errorbar(cat_flx['Wavelength'][j], cat_flx['FluxCat'][j]*jy_to_mujy, xerr=cat_flx['Wavelength'][j]*error_w, yerr=cat_flx['FluxErrTot'][j]*jy_to_mujy, ls='', capsize=0.5, capthick=0.5, elinewidth=error_lw, color='indigo', label='B')
            if (cat_flx['Wavelength'][j] > 5959.323) & (cat_flx['Wavelength'][j] < 5959.325):
                h2 = plt.errorbar(cat_flx['Wavelength'][j], cat_flx['FluxCat'][j]*jy_to_mujy, xerr=cat_flx['Wavelength'][j]*error_w, yerr=cat_flx['FluxErrTot'][j]*jy_to_mujy, ls='', capsize=0.5, capthick=0.5, elinewidth=error_lw, color='darkblue', label='V')
            if (cat_flx['Wavelength'][j] > 7704.696) & (cat_flx['Wavelength'][j] < 7704.698):
                h3 = plt.errorbar(cat_flx['Wavelength'][j], cat_flx['FluxCat'][j]*jy_to_mujy, xerr=cat_flx['Wavelength'][j]*error_w, yerr=cat_flx['FluxErrTot'][j]*jy_to_mujy, ls='', capsize=0.5, capthick=0.5, elinewidth=error_lw, color='blue', label='i')
            if cat_flx['Wavelength'][j] == 9.048604e+03:
                h4 = plt.errorbar(cat_flx['Wavelength'][j], cat_flx['FluxCat'][j]*jy_to_mujy, xerr=cat_flx['Wavelength'][j]*error_w, yerr=cat_flx['FluxErrTot'][j]*jy_to_mujy, ls='', capsize=0.5, capthick=0.5, elinewidth=error_lw, color='darkcyan', label='z')
            if cat_flx['Wavelength'][j] == 1.057723e+04:
                h5 = plt.errorbar(cat_flx['Wavelength'][j], cat_flx['FluxCat'][j]*jy_to_mujy, xerr=cat_flx['Wavelength'][j]*error_w, yerr=cat_flx['FluxErrTot'][j]*jy_to_mujy, ls='', capsize=0.5, capthick=0.5, elinewidth=error_lw, color='olivedrab', label='Y')
            if cat_flx['Wavelength'][j] == 1.250085e+04:
                h6 = plt.errorbar(cat_flx['Wavelength'][j], cat_flx['FluxCat'][j]*jy_to_mujy, xerr=cat_flx['Wavelength'][j]*error_w, yerr=cat_flx['FluxErrTot'][j]*jy_to_mujy, ls='', capsize=0.5, capthick=0.5, elinewidth=error_lw, color='yellowgreen', label='J')
            if cat_flx['Wavelength'][j] == 1.541899e+04:
                h7 = plt.errorbar(cat_flx['Wavelength'][j], cat_flx['FluxCat'][j]*jy_to_mujy, xerr=cat_flx['Wavelength'][j]*error_w, yerr=cat_flx['FluxErrTot'][j]*jy_to_mujy, ls='', capsize=0.5, capthick=0.5, elinewidth=error_lw, color='orange', label='H')
            if cat_flx['Wavelength'][j] == 2.154491e+04:
                h8 = plt.errorbar(cat_flx['Wavelength'][j], cat_flx['FluxCat'][j]*jy_to_mujy, xerr=cat_flx['Wavelength'][j]*error_w, yerr=cat_flx['FluxErrTot'][j]*jy_to_mujy, ls='', capsize=0.5, capthick=0.5, elinewidth=error_lw, color='orangered', label='K')
            if cat_flx['Wavelength'][j] == 3.563426e+04:
                h9 = plt.errorbar(cat_flx['Wavelength'][j], cat_flx['FluxCat'][j]*jy_to_mujy, xerr=cat_flx['Wavelength'][j]*error_w, yerr=cat_flx['FluxErrTot'][j]*jy_to_mujy, ls='', capsize=0.5, capthick=0.5, elinewidth=error_lw, color='red', label='3.6mu')
            if cat_flx['Wavelength'][j] == 4.511019e+04:
                h10 = plt.errorbar(cat_flx['Wavelength'][j], cat_flx['FluxCat'][j]*jy_to_mujy, xerr=cat_flx['Wavelength'][j]*error_w, yerr=cat_flx['FluxErrTot'][j]*jy_to_mujy, ls='', capsize=0.5, capthick=0.5, elinewidth=error_lw, color='firebrick', label='4.5mu')
        if i==526:
            plt.legend(handles=[h1,h2,h3,h4,h6,h7,h8,h9,h10],frameon=False,numpoints=1,fontsize=7,loc='upper left')
        else:
            plt.legend(handles=[h1,h2,h3,h4,h5,h6,h7,h8,h9,h10],frameon=False,numpoints=1,fontsize=7,loc='upper left')
        plt.xscale('log')
        plt.yscale('log')
        plt.xlabel('$\lambda$ in $\AA$')
        plt.ylabel('$F_{\\nu}$ in $\mu$Jy')
        for j in range(len(cat_z)):
            if cat_z['ID'][j]==i+1:
                plt.text(s='$z_{phot}='+'{:.2f}'.format(cat_z['z_a'][j])+'$',x=10000, y=200)
        path = '/Users/justin/Documents/Master_Thesis/presentation/plots/eazy/sep/sed_' + str(i+1) + 'add.pdf'
        plt.savefig(path, format='pdf', dpi=1000,bbox_inches='tight')
        plt.clf()
        
        g=plt.figure(figsize=(4,4))
        ax=g.add_subplot(111)
        plt.plot(cat_chi['z'],cat_chi['Chi^2'], marker='', lw=1., ls='-', color='black',label='')
        #plt.yscale('log')
        x1,x2,y1,y2 = plt.axis()
        plt.axis((0,11,0,20))
        for j in range(len(cat_z)):
            if cat_z['ID'][j]==i+1:
                chi_l1=np.interp(cat_z['l1'][j],cat_chi['z'],cat_chi['Chi^2'])
                chi_u1=np.interp(cat_z['u1'][j],cat_chi['z'],cat_chi['Chi^2'])
                chi_l2=np.interp(cat_z['l2'][j],cat_chi['z'],cat_chi['Chi^2'])
                chi_u2=np.interp(cat_z['u2'][j],cat_chi['z'],cat_chi['Chi^2'])
                chi_l3=np.interp(cat_z['l3'][j],cat_chi['z'],cat_chi['Chi^2'])
                chi_u3=np.interp(cat_z['u3'][j],cat_chi['z'],cat_chi['Chi^2'])
                chi_best=np.interp(cat_z['z_a'][j],cat_chi['z'],cat_chi['Chi^2'])
                plt.plot([cat_z['l1'][j],cat_z['l1'][j]], [0,chi_l1], marker='', lw=0.5, ls='-', color='black',label='$1\sigma$')
                plt.plot([cat_z['l2'][j],cat_z['l2'][j]], [0,chi_l2], marker='', lw=0.5, ls='--', color='black',label='$2\sigma$')
                plt.plot([cat_z['l3'][j],cat_z['l3'][j]], [0,chi_l3], marker='', lw=0.5, ls='-.', color='black',label='$3\sigma$')
                plt.plot([cat_z['u1'][j],cat_z['u1'][j]], [0,chi_u1], marker='', lw=0.5, ls='-', color='black')
                plt.plot([cat_z['u2'][j],cat_z['u2'][j]], [0,chi_u2], marker='', lw=0.5, ls='--', color='black')
                plt.plot([cat_z['u3'][j],cat_z['u3'][j]], [0,chi_u3], marker='', lw=0.5, ls='-.', color='black')
                #plt.arrow(cat_z['z_a'][j], chi_best, marker='.', ms=6, ls='', color='red')
                ax.arrow(cat_z['z_a'][j], chi_best+3, 0., -2,length_includes_head=True, head_width=0.2, head_length=0.2, fc='k', ec='k', lw=1)
        plt.fill_between(cat_chi['z'],0,cat_chi['Chi^2'],color='lightgray')
        plt.xlabel('$\mathrm{z}$')
        plt.ylabel('$\mathrm{\chi^2}$')
        plt.legend(frameon=False, loc='best')
        #for label in (ax.get_xticklabels() + ax.get_yticklabels()):
            #label.set_fontsize(7)
        path = '/Users/justin/Documents/Master_Thesis/presentation/plots/eazy/sep/chi2_' + str(i+1) + 'add.pdf'
        plt.savefig(path, format='pdf', dpi=1000,bbox_inches='tight')
        plt.clf()

        h=plt.figure(figsize=(4,4))
        ax=h.add_subplot(111)
        plt.plot(cat_chi['z'],cat_chi['Chi^2'], marker='', lw=1., ls='-', color='black',label='')
        plt.yscale('log')
        x1,x2,y1,y2 = plt.axis()
        plt.axis((0,11,y1,y2))
        for j in range(len(cat_z)):
            if cat_z['ID'][j]==i+1:
                chi_l1=np.interp(cat_z['l1'][j],cat_chi['z'],cat_chi['Chi^2'])
                chi_u1=np.interp(cat_z['u1'][j],cat_chi['z'],cat_chi['Chi^2'])
                chi_l2=np.interp(cat_z['l2'][j],cat_chi['z'],cat_chi['Chi^2'])
                chi_u2=np.interp(cat_z['u2'][j],cat_chi['z'],cat_chi['Chi^2'])
                chi_l3=np.interp(cat_z['l3'][j],cat_chi['z'],cat_chi['Chi^2'])
                chi_u3=np.interp(cat_z['u3'][j],cat_chi['z'],cat_chi['Chi^2'])
                chi_best=np.interp(cat_z['z_a'][j],cat_chi['z'],cat_chi['Chi^2'])
                plt.plot([cat_z['l1'][j],cat_z['l1'][j]], [y1,chi_l1], marker='', lw=0.5, ls='-', color='black',label='$1\sigma$')
                plt.plot([cat_z['l2'][j],cat_z['l2'][j]], [y1,chi_l2], marker='', lw=0.5, ls='--', color='black',label='$2\sigma$')
                plt.plot([cat_z['l3'][j],cat_z['l3'][j]], [y1,chi_l3], marker='', lw=0.5, ls='-.', color='black',label='$3\sigma$')
                plt.plot([cat_z['u1'][j],cat_z['u1'][j]], [y1,chi_u1], marker='', lw=0.5, ls='-', color='black')
                plt.plot([cat_z['u2'][j],cat_z['u2'][j]], [y1,chi_u2], marker='', lw=0.5, ls='--', color='black')
                plt.plot([cat_z['u3'][j],cat_z['u3'][j]], [y1,chi_u3], marker='', lw=0.5, ls='-.', color='black')
                #plt.arrow(cat_z['z_a'][j], chi_best, marker='.', ms=6, ls='', color='red')
                ax.arrow(cat_z['z_a'][j], chi_best*7, 0., -chi_best*4.,length_includes_head=True, head_width=0.2, head_length=chi_best*0.3, fc='k', ec='k', lw=1)
        plt.fill_between(cat_chi['z'],y1,cat_chi['Chi^2'],color='lightgray')
        plt.xlabel('$\mathrm{z}$')
        plt.ylabel('$\mathrm{\chi^2}$')
        plt.legend(frameon=False, loc='best')
        #for label in (ax.get_xticklabels() + ax.get_yticklabels()):
            #label.set_fontsize(7)
        path = '/Users/justin/Documents/Master_Thesis/presentation/plots/eazy/sep/chi2_' + str(i+1) + '_logadd.pdf'
        plt.savefig(path, format='pdf', dpi=1000,bbox_inches='tight')
        plt.clf()

In [8]:
if part==6: #same as part 5, but combining manual and automatic photometry plots for sed and chi2
    path = '/Users/justin/Documents/Master_Thesis/data/catalogues/chandra_7ms/big_table.fits'
    big_table = fits.open(path)[1].data
    error_lw = 1.
    error_w = 0.03
    path = '/Users/justin/Documents/Master_Thesis/softwares/eazy/inputs/OUTPUT_manual/photz.zout'
    cat_z = pd.read_csv(path, delim_whitespace=True, header=None, names=['ID', 'z_spec', 'z_a','f','chi2','l1','u1','l2','u2','l3','u3','3f','s2s','gt4','4f','bt'], skiprows=[0,1])
   
    for i in [526,621,639,713]:
        path_sed = '/Users/justin/Documents/Master_Thesis/softwares/eazy/inputs/OUTPUT_manual/photz_' + str(i+1) + '.temp_sed'
        path_flx = '/Users/justin/Documents/Master_Thesis/softwares/eazy/inputs/OUTPUT_manual/photz_' + str(i+1) + '.obs_sed'
        path_chi = '/Users/justin/Documents/Master_Thesis/softwares/eazy/inputs/OUTPUT_manual/photz_' + str(i+1) + '.pz'
        cat_sed = pd.read_csv(path_sed, delim_whitespace=True, header=None, names=['Wavelength', 'Flux'], skiprows=[0,1])
        cat_flx = pd.read_csv(path_flx, delim_whitespace=True, header=None, names=['Wavelength', 'FluxCat', 'FluxErrCat', 'FluxErrTot', 'FluxTemp'], skiprows=1)
        cat_chi = pd.read_csv(path_chi, delim_whitespace=True, header=None, names=['z', 'Chi^2'], skiprows=1)

        jy_to_mujy = 1e6
        
        path_add=['','_manual']
        f = plt.figure(figsize=(4,6))
        for nplots in [0,1]:
            path_sed = '/Users/justin/Documents/Master_Thesis/softwares/eazy/inputs/OUTPUT' + path_add[nplots] + '/photz_' + str(i+1) + '.temp_sed'
            path_flx = '/Users/justin/Documents/Master_Thesis/softwares/eazy/inputs/OUTPUT' + path_add[nplots] + '/photz_' + str(i+1) + '.obs_sed'
            path_chi = '/Users/justin/Documents/Master_Thesis/softwares/eazy/inputs/OUTPUT' + path_add[nplots] + '/photz_' + str(i+1) + '.pz'
            path = '/Users/justin/Documents/Master_Thesis/softwares/eazy/inputs/OUTPUT' + path_add[nplots] + '/photz.zout'
            cat_sed = pd.read_csv(path_sed, delim_whitespace=True, header=None, names=['Wavelength', 'Flux'], skiprows=[0,1])
            cat_flx = pd.read_csv(path_flx, delim_whitespace=True, header=None, names=['Wavelength', 'FluxCat', 'FluxErrCat', 'FluxErrTot', 'FluxTemp'], skiprows=1)
            cat_chi = pd.read_csv(path_chi, delim_whitespace=True, header=None, names=['z', 'Chi^2'], skiprows=1)
            cat_z = pd.read_csv(path, delim_whitespace=True, header=None, names=['ID', 'z_spec', 'z_a','f','chi2','l1','u1','l2','u2','l3','u3','3f','s2s','gt4','4f','bt'], skiprows=[0,1])

            if nplots == 0:
                ax = f.add_subplot(211)
            else:
                ax = f.add_subplot(212)
            ax.plot(cat_sed['Wavelength'], cat_sed['Flux']*jy_to_mujy, marker='', lw=0.5, ls='-', color='dimgray')
            plt.axis([3e3, 5e4, 9e-10*jy_to_mujy, 5e-4*jy_to_mujy])
            #ax.set_aspect(0.25)
            for j in range(len(cat_flx)):
                if cat_flx['Wavelength'][j] == 4328.683:
                    h1 = plt.errorbar(cat_flx['Wavelength'][j], cat_flx['FluxCat'][j]*jy_to_mujy, xerr=cat_flx['Wavelength'][j]*error_w, yerr=cat_flx['FluxErrTot'][j]*jy_to_mujy, ls='', capsize=0.5, capthick=0.5, elinewidth=error_lw, color='indigo', label='B')
                if (cat_flx['Wavelength'][j] > 5959.323) & (cat_flx['Wavelength'][j] < 5959.325):
                    h2 = plt.errorbar(cat_flx['Wavelength'][j], cat_flx['FluxCat'][j]*jy_to_mujy, xerr=cat_flx['Wavelength'][j]*error_w, yerr=cat_flx['FluxErrTot'][j]*jy_to_mujy, ls='', capsize=0.5, capthick=0.5, elinewidth=error_lw, color='darkblue', label='V')
                if (cat_flx['Wavelength'][j] > 7704.696) & (cat_flx['Wavelength'][j] < 7704.698):
                    h3 = plt.errorbar(cat_flx['Wavelength'][j], cat_flx['FluxCat'][j]*jy_to_mujy, xerr=cat_flx['Wavelength'][j]*error_w, yerr=cat_flx['FluxErrTot'][j]*jy_to_mujy, ls='', capsize=0.5, capthick=0.5, elinewidth=error_lw, color='blue', label='i')
                if cat_flx['Wavelength'][j] == 9.048604e+03:
                    h4 = plt.errorbar(cat_flx['Wavelength'][j], cat_flx['FluxCat'][j]*jy_to_mujy, xerr=cat_flx['Wavelength'][j]*error_w, yerr=cat_flx['FluxErrTot'][j]*jy_to_mujy, ls='', capsize=0.5, capthick=0.5, elinewidth=error_lw, color='darkcyan', label='z')
                if cat_flx['Wavelength'][j] == 1.057723e+04:
                    h5 = plt.errorbar(cat_flx['Wavelength'][j], cat_flx['FluxCat'][j]*jy_to_mujy, xerr=cat_flx['Wavelength'][j]*error_w, yerr=cat_flx['FluxErrTot'][j]*jy_to_mujy, ls='', capsize=0.5, capthick=0.5, elinewidth=error_lw, color='olivedrab', label='Y')
                if cat_flx['Wavelength'][j] == 1.250085e+04:
                    h6 = plt.errorbar(cat_flx['Wavelength'][j], cat_flx['FluxCat'][j]*jy_to_mujy, xerr=cat_flx['Wavelength'][j]*error_w, yerr=cat_flx['FluxErrTot'][j]*jy_to_mujy, ls='', capsize=0.5, capthick=0.5, elinewidth=error_lw, color='yellowgreen', label='J')
                if cat_flx['Wavelength'][j] == 1.541899e+04:
                    h7 = plt.errorbar(cat_flx['Wavelength'][j], cat_flx['FluxCat'][j]*jy_to_mujy, xerr=cat_flx['Wavelength'][j]*error_w, yerr=cat_flx['FluxErrTot'][j]*jy_to_mujy, ls='', capsize=0.5, capthick=0.5, elinewidth=error_lw, color='orange', label='H')
                if cat_flx['Wavelength'][j] == 2.154491e+04:
                    h8 = plt.errorbar(cat_flx['Wavelength'][j], cat_flx['FluxCat'][j]*jy_to_mujy, xerr=cat_flx['Wavelength'][j]*error_w, yerr=cat_flx['FluxErrTot'][j]*jy_to_mujy, ls='', capsize=0.5, capthick=0.5, elinewidth=error_lw, color='orangered', label='K')
                if cat_flx['Wavelength'][j] == 3.563426e+04:
                    h9 = plt.errorbar(cat_flx['Wavelength'][j], cat_flx['FluxCat'][j]*jy_to_mujy, xerr=cat_flx['Wavelength'][j]*error_w, yerr=cat_flx['FluxErrTot'][j]*jy_to_mujy, ls='', capsize=0.5, capthick=0.5, elinewidth=error_lw, color='red', label='3.6mu')
                if cat_flx['Wavelength'][j] == 4.511019e+04:
                    h10 = plt.errorbar(cat_flx['Wavelength'][j], cat_flx['FluxCat'][j]*jy_to_mujy, xerr=cat_flx['Wavelength'][j]*error_w, yerr=cat_flx['FluxErrTot'][j]*jy_to_mujy, ls='', capsize=0.5, capthick=0.5, elinewidth=error_lw, color='firebrick', label='4.5mu')
            if i==526:
                plt.legend(handles=[h1,h2,h3,h4,h6,h7,h8,h9,h10],frameon=False,numpoints=1,fontsize=7,loc='upper left')
            else:
                plt.legend(handles=[h1,h2,h3,h4,h5,h6,h7,h8,h9,h10],frameon=False,numpoints=1,fontsize=7,loc='upper left')
            plt.xscale('log')
            plt.yscale('log')
            if nplots == 1:
                plt.xlabel('$\lambda$ in $\AA$')
                plt.ylabel('$F_{\\nu}$ in $\mu$Jy')
            if nplots == 0:
                plt.setp(ax.get_xticklabels(), visible=False)
            for j in range(len(cat_z)):
                if cat_z['ID'][j]==i+1:
                    plt.text(s='$z_{phot}='+'{:.2f}'.format(cat_z['z_a'][j])+'$',x=10000, y=200)
        plt.subplots_adjust(hspace = 0.)
        path = '/Users/justin/Documents/Master_Thesis/presentation/plots/eazy/sep/sed_' + str(i+1) + 'combined.pdf'
        plt.savefig(path, format='pdf', dpi=1000,bbox_inches='tight')
        plt.clf()
        
        g=plt.figure(figsize=(4,6))
        for nplots in [0,1]:
            path_sed = '/Users/justin/Documents/Master_Thesis/softwares/eazy/inputs/OUTPUT' + path_add[nplots] + '/photz_' + str(i+1) + '.temp_sed'
            path_flx = '/Users/justin/Documents/Master_Thesis/softwares/eazy/inputs/OUTPUT' + path_add[nplots] + '/photz_' + str(i+1) + '.obs_sed'
            path_chi = '/Users/justin/Documents/Master_Thesis/softwares/eazy/inputs/OUTPUT' + path_add[nplots] + '/photz_' + str(i+1) + '.pz'
            path = '/Users/justin/Documents/Master_Thesis/softwares/eazy/inputs/OUTPUT' + path_add[nplots] + '/photz.zout'
            cat_sed = pd.read_csv(path_sed, delim_whitespace=True, header=None, names=['Wavelength', 'Flux'], skiprows=[0,1])
            cat_flx = pd.read_csv(path_flx, delim_whitespace=True, header=None, names=['Wavelength', 'FluxCat', 'FluxErrCat', 'FluxErrTot', 'FluxTemp'], skiprows=1)
            cat_chi = pd.read_csv(path_chi, delim_whitespace=True, header=None, names=['z', 'Chi^2'], skiprows=1)
            cat_z = pd.read_csv(path, delim_whitespace=True, header=None, names=['ID', 'z_spec', 'z_a','f','chi2','l1','u1','l2','u2','l3','u3','3f','s2s','gt4','4f','bt'], skiprows=[0,1])

            if nplots == 0:
                ax = g.add_subplot(211)
            else:
                ax = g.add_subplot(212)
            plt.plot(cat_chi['z'],cat_chi['Chi^2'], marker='', lw=1., ls='-', color='black',label='')
            #plt.yscale('log')
            x1,x2,y1,y2 = plt.axis()
            plt.axis((0,11,0,20))
            for j in range(len(cat_z)):
                if cat_z['ID'][j]==i+1:
                    chi_l1=np.interp(cat_z['l1'][j],cat_chi['z'],cat_chi['Chi^2'])
                    chi_u1=np.interp(cat_z['u1'][j],cat_chi['z'],cat_chi['Chi^2'])
                    chi_l2=np.interp(cat_z['l2'][j],cat_chi['z'],cat_chi['Chi^2'])
                    chi_u2=np.interp(cat_z['u2'][j],cat_chi['z'],cat_chi['Chi^2'])
                    chi_l3=np.interp(cat_z['l3'][j],cat_chi['z'],cat_chi['Chi^2'])
                    chi_u3=np.interp(cat_z['u3'][j],cat_chi['z'],cat_chi['Chi^2'])
                    chi_best=np.interp(cat_z['z_a'][j],cat_chi['z'],cat_chi['Chi^2'])
                    plt.plot([cat_z['l1'][j],cat_z['l1'][j]], [0,chi_l1], marker='', lw=0.5, ls='-', color='black',label='$1\sigma$')
                    plt.plot([cat_z['l2'][j],cat_z['l2'][j]], [0,chi_l2], marker='', lw=0.5, ls='--', color='black',label='$2\sigma$')
                    plt.plot([cat_z['l3'][j],cat_z['l3'][j]], [0,chi_l3], marker='', lw=0.5, ls='-.', color='black',label='$3\sigma$')
                    plt.plot([cat_z['u1'][j],cat_z['u1'][j]], [0,chi_u1], marker='', lw=0.5, ls='-', color='black')
                    plt.plot([cat_z['u2'][j],cat_z['u2'][j]], [0,chi_u2], marker='', lw=0.5, ls='--', color='black')
                    plt.plot([cat_z['u3'][j],cat_z['u3'][j]], [0,chi_u3], marker='', lw=0.5, ls='-.', color='black')
                    #plt.arrow(cat_z['z_a'][j], chi_best, marker='.', ms=6, ls='', color='red')
                    ax.arrow(cat_z['z_a'][j], chi_best+3, 0., -2,length_includes_head=True, head_width=0.2, head_length=0.2, fc='k', ec='k', lw=1)
            plt.fill_between(cat_chi['z'],0,cat_chi['Chi^2'],color='lightgray')
            if nplots ==1:
                plt.xlabel('$\mathrm{z}$')
                plt.ylabel('$\mathrm{\chi^2}$')
            plt.legend(frameon=False, loc='best')
            if nplots == 0:
                plt.setp(ax.get_xticklabels(), visible=False)
            #for label in (ax.get_xticklabels() + ax.get_yticklabels()):
                #label.set_fontsize(7)
        plt.subplots_adjust(hspace = 0.)
        path = '/Users/justin/Documents/Master_Thesis/presentation/plots/eazy/sep/chi2_' + str(i+1) + 'combined.pdf'
        plt.savefig(path, format='pdf', dpi=1000,bbox_inches='tight')
        plt.clf()

        h=plt.figure(figsize=(4,6))
        for nplots in [0,1]:
            path_sed = '/Users/justin/Documents/Master_Thesis/softwares/eazy/inputs/OUTPUT' + path_add[nplots] + '/photz_' + str(i+1) + '.temp_sed'
            path_flx = '/Users/justin/Documents/Master_Thesis/softwares/eazy/inputs/OUTPUT' + path_add[nplots] + '/photz_' + str(i+1) + '.obs_sed'
            path_chi = '/Users/justin/Documents/Master_Thesis/softwares/eazy/inputs/OUTPUT' + path_add[nplots] + '/photz_' + str(i+1) + '.pz'
            path = '/Users/justin/Documents/Master_Thesis/softwares/eazy/inputs/OUTPUT' + path_add[nplots] + '/photz.zout'
            cat_sed = pd.read_csv(path_sed, delim_whitespace=True, header=None, names=['Wavelength', 'Flux'], skiprows=[0,1])
            cat_flx = pd.read_csv(path_flx, delim_whitespace=True, header=None, names=['Wavelength', 'FluxCat', 'FluxErrCat', 'FluxErrTot', 'FluxTemp'], skiprows=1)
            cat_chi = pd.read_csv(path_chi, delim_whitespace=True, header=None, names=['z', 'Chi^2'], skiprows=1)
            cat_z = pd.read_csv(path, delim_whitespace=True, header=None, names=['ID', 'z_spec', 'z_a','f','chi2','l1','u1','l2','u2','l3','u3','3f','s2s','gt4','4f','bt'], skiprows=[0,1])

            if nplots == 0:
                ax = h.add_subplot(211)
            else:
                ax = h.add_subplot(212)
            plt.plot(cat_chi['z'],cat_chi['Chi^2'], marker='', lw=1., ls='-', color='black',label='')
            plt.yscale('log')
            x1,x2,y1,y2 = plt.axis()
            plt.axis((0,11,y1,y2))
            for j in range(len(cat_z)):
                if cat_z['ID'][j]==i+1:
                    chi_l1=np.interp(cat_z['l1'][j],cat_chi['z'],cat_chi['Chi^2'])
                    chi_u1=np.interp(cat_z['u1'][j],cat_chi['z'],cat_chi['Chi^2'])
                    chi_l2=np.interp(cat_z['l2'][j],cat_chi['z'],cat_chi['Chi^2'])
                    chi_u2=np.interp(cat_z['u2'][j],cat_chi['z'],cat_chi['Chi^2'])
                    chi_l3=np.interp(cat_z['l3'][j],cat_chi['z'],cat_chi['Chi^2'])
                    chi_u3=np.interp(cat_z['u3'][j],cat_chi['z'],cat_chi['Chi^2'])
                    chi_best=np.interp(cat_z['z_a'][j],cat_chi['z'],cat_chi['Chi^2'])
                    plt.plot([cat_z['l1'][j],cat_z['l1'][j]], [y1,chi_l1], marker='', lw=0.5, ls='-', color='black',label='$1\sigma$')
                    plt.plot([cat_z['l2'][j],cat_z['l2'][j]], [y1,chi_l2], marker='', lw=0.5, ls='--', color='black',label='$2\sigma$')
                    plt.plot([cat_z['l3'][j],cat_z['l3'][j]], [y1,chi_l3], marker='', lw=0.5, ls='-.', color='black',label='$3\sigma$')
                    plt.plot([cat_z['u1'][j],cat_z['u1'][j]], [y1,chi_u1], marker='', lw=0.5, ls='-', color='black')
                    plt.plot([cat_z['u2'][j],cat_z['u2'][j]], [y1,chi_u2], marker='', lw=0.5, ls='--', color='black')
                    plt.plot([cat_z['u3'][j],cat_z['u3'][j]], [y1,chi_u3], marker='', lw=0.5, ls='-.', color='black')
                    #plt.arrow(cat_z['z_a'][j], chi_best, marker='.', ms=6, ls='', color='red')
                    ax.arrow(cat_z['z_a'][j], chi_best*7, 0., -chi_best*4.,length_includes_head=True, head_width=0.2, head_length=chi_best*0.3, fc='k', ec='k', lw=1)
            plt.fill_between(cat_chi['z'],y1,cat_chi['Chi^2'],color='lightgray')
            if nplots ==1:
                plt.xlabel('$\mathrm{z}$')
                plt.ylabel('$\mathrm{\chi^2}$')
            plt.legend(frameon=False, loc='best')
            if nplots == 0:
                plt.setp(ax.get_xticklabels(), visible=False)
            #for label in (ax.get_xticklabels() + ax.get_yticklabels()):
                #label.set_fontsize(7)
        plt.subplots_adjust(hspace = 0.)
        path = '/Users/justin/Documents/Master_Thesis/presentation/plots/eazy/sep/chi2_' + str(i+1) + '_logcombined.pdf'
        plt.savefig(path, format='pdf', dpi=1000,bbox_inches='tight')
        plt.clf()